In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Input
from keras.callbacks import ModelCheckpoint

In [2]:
print(tf.__version__)

2.10.0


In [3]:
#  TODO: Add functionality to handle joining multiple csvs together from different file locations into usable df

In [22]:
#  Example df with correct columns

ex_path = "C:/Users/Eric/Downloads/emg_dataframe.csv"
real_path = "C:/Users/Eric/Downloads/1738796078.0934057.csv"

df = pd.read_csv(real_path, header=None)
df.columns = df.iloc[0]
df = df.drop(0, axis=0).reset_index(drop=True)

#rand_data = np.random.rand(1000, len(df.columns))
#df_rand = pd.DataFrame(rand_data, columns=df.columns)

#df = df.dropna(axis=1, how='all')
#df_rand = df_rand.dropna(axis=1, how='all')

# Concatenate the dataframes
#df = pd.concat([df, df_rand], ignore_index=True)

C:\Users\Eric\AppData\Local\Temp\ipykernel_7056\4056852923.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(real_path, header=None)


In [23]:
#just for reference rn
data = {
    'row': [0],
    'time': [1, 10, 19, 28, 37],
    'emg': [2, 11, 20, 29, 38],
    'imp': [3, 12, 21, 30, 39],
    'accx': [4, 13, 22, 31, 40],
    'accy': [5, 14, 23, 32, 41],
    'accz': [6, 15, 24, 33, 42],
    'gyrx': [7, 16, 25, 34, 43],
    'gyry': [8, 17, 26, 35, 44],
    'gyrz': [9, 18, 27, 36, 45],
}
start_row = 110
end_row_emg = 396800
end_row_other = 46700

In [24]:
# Drop unnecessary columns
columns_to_drop = [col for col in df.columns if isinstance(col, str) and ('IMP' in col.upper() or 'TIME' in col.upper()) and col != 'time 0']
df = df.drop(columns=columns_to_drop)
df = df.drop(df.columns[0], axis=1)
df = df.apply(pd.to_numeric, errors='coerce')

In [25]:
df = df[200:390000]

In [26]:
df.columns

Index(['time 0', 'sensor 0 EMG 1 (1259.259 Hz)', 'sensor 0 ACC X (148.148 Hz)',
       'sensor 0 ACC Y (148.148 Hz)', 'sensor 0 ACC Z (148.148 Hz)',
       'sensor 0 GYRO X (148.148 Hz)', 'sensor 0 GYRO Y (148.148 Hz)',
       'sensor 0 GYRO Z (148.148 Hz)', 'sensor 1 EMG 1 (1259.259 Hz)',
       'sensor 1 ACC X (148.148 Hz)', 'sensor 1 ACC Y (148.148 Hz)',
       'sensor 1 ACC Z (148.148 Hz)', 'sensor 1 GYRO X (148.148 Hz)',
       'sensor 1 GYRO Y (148.148 Hz)', 'sensor 1 GYRO Z (148.148 Hz)',
       'sensor 2 EMG 1 (1259.259 Hz)', 'sensor 2 ACC X (148.148 Hz)',
       'sensor 2 ACC Y (148.148 Hz)', 'sensor 2 ACC Z (148.148 Hz)',
       'sensor 2 GYRO X (148.148 Hz)', 'sensor 2 GYRO Y (148.148 Hz)',
       'sensor 2 GYRO Z (148.148 Hz)', 'sensor 3 EMG 1 (1259.259 Hz)',
       'sensor 3 ACC X (148.148 Hz)', 'sensor 3 ACC Y (148.148 Hz)',
       'sensor 3 ACC Z (148.148 Hz)', 'sensor 3 GYRO X (148.148 Hz)',
       'sensor 3 GYRO Y (148.148 Hz)', 'sensor 3 GYRO Z (148.148 Hz)',
       '

In [27]:
emg_columns = ['sensor 0 EMG 1 (1259.259 Hz)',
               'sensor 1 EMG 1 (1259.259 Hz)',
               'sensor 2 EMG 1 (1259.259 Hz)',
               'sensor 3 EMG 1 (1259.259 Hz)',
               'sensor 4 EMG 1 (1259.259 Hz)',
               'sensor 5 EMG 1 (1259.259 Hz)']

In [28]:
def resample_data(df, target_fs):
    """
    Resample data to the target sampling frequency.
    
    Parameters:
    - df: The DataFrame containing the time and signal data (e.g., 'time 0', EMG, GYRO, ACC columns).
    - target_fs: The target sampling frequency (Hz).
    
    Returns:
    - df_resampled: A new DataFrame with all columns resampled to the target frequency.
    """
    # Ensure 'time 0' is the time column and handle NaN values
    time_column = df['time 0'].dropna()  # Remove NaN values
    if len(time_column) < 2:
        raise ValueError("Time column 'time 0' has fewer than 2 valid entries after removing NaN.")
    
    # Calculate total duration and original sampling frequency
    total_duration = time_column.iloc[-1] - time_column.iloc[0]  # Total time span
    if np.isnan(total_duration):
        raise ValueError("Total duration is NaN. Check 'time 0' for invalid values.")
    
    original_length = len(time_column)  # Number of valid samples in time column
    original_fs = original_length / total_duration  # Average original sampling frequency (Hz)
    
    # Calculate target length based on target_fs
    target_length = int(total_duration * target_fs)  # Number of samples at target frequency
    if target_length <= 0:
        raise ValueError(f"Target length {target_length} is invalid. Check total_duration and target_fs.")
    
    new_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], target_length)  # New time array
    
    # Initialize resampled DataFrame with the new time column
    df_resampled = pd.DataFrame({'time 0': new_time})
    
    # Identify signal columns
    emg_columns = [col for col in df.columns if 'EMG' in col]
    gyro_accel_columns = [col for col in df.columns if 'GYRO' in col or 'ACC' in col]
    
    # Resample all signal columns to the target length
    for col in emg_columns + gyro_accel_columns:
        original_signal = df[col].dropna()  # Remove NaN values from signal
        if len(original_signal) < 2:
            print(f"Warning: Column '{col}' has fewer than 2 valid entries. Filling with zeros.")
            df_resampled[col] = np.zeros(target_length)
            continue
        
        # Interpolate to match time column length if needed
        if len(original_signal) != original_length:
            original_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], len(original_signal))
            interpolated_signal = np.interp(time_column, original_time, original_signal)
            resampled_signal = signal.resample(interpolated_signal, target_length)
        else:
            resampled_signal = signal.resample(original_signal, target_length)
        df_resampled[col] = resampled_signal
    
    return df_resampled

In [29]:
#  Implement butterworth filters of different orders, cutoff frequencies, sampling frequencies, and filter_band type to columns of a dataframe.

def butterworth_filter(data, order, cutoff, fs, filter_band='low'):
    nyquist = 0.5 * fs
    normalized_cutoff = np.array(cutoff) / nyquist
    
    #  Create Butterworth filter coefficients
    if filter_band == 'low':
        b, a = signal.butter(order, normalized_cutoff[0], btype='low')
    elif filter_band == 'high':
        b, a = signal.butter(order, normalized_cutoff[0], btype='high')
    elif filter_band == 'bandpass':
        b, a = signal.butter(order, normalized_cutoff, btype='bandpass')
    elif filter_band == 'bandstop':
        b, a = signal.butter(order, normalized_cutoff, btype='bandstop')
    else:
        raise ValueError(f"unsupported filter: {filter_band}")
    
    #  Apply filter
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

def apply_filter(df, columns, order=4, cutoff=[1.0], fs=1000, filter_band='low'):
    '''
    Apply a Butterworth filter of specified order and cutoff to the given columns of the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the filter to (e.g., EMG columns).
    - order: The order of the filter.
    - cutoff: The cutoff frequency or frequencies.
    - fs: Sampling frequency of the signal.
    - filter_band: Type of filter ('low', 'high', 'bandpass', 'bandstop').
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    for col in columns:
        df[col] = butterworth_filter(df[col].values, order, cutoff, fs, filter_band)
    
    return df

In [30]:
def apply_notch(df, columns, notch_freq=60, fs=1000, Q=30):
    '''
    Apply a notch filter to remove a specific frequency (e.g., powerline noise at 60Hz)
    to the specified columns in the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the notch filter to.
    - notch_freq: Frequency to remove with the notch filter (e.g., 60Hz for powerline noise).
    - fs: Sampling frequency of the signal.
    - Q: Quality factor for the notch filter.
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    nyquist = 0.5 * fs
    normalized_frequency = notch_freq / nyquist

    b, a = signal.iirnotch(normalized_frequency, Q)

    for col in columns:
        df[col] = signal.filtfilt(b, a, df[col].values)
    
    return df

In [31]:
def split_dataframe(df):
    time_column = df.columns[0]
    
    sensor_columns = df.columns[1:]

    left_keywords = ['sensor 0', 'sensor 1', 'sensor 2']

    left_sensors = [col for col in sensor_columns if any(keyword in col for keyword in left_keywords)]
    right_sensors = [col for col in sensor_columns if col not in left_sensors]

    left_emg = [col for col in left_sensors if 'EMG' in col]
    right_emg = [col for col in right_sensors if 'EMG' in col]

    left_inputs = [col for col in left_sensors if col not in left_emg]
    right_inputs = [col for col in right_sensors if col not in right_emg]

    return df[left_inputs], df[left_emg], df[right_inputs], df[right_emg]

In [32]:
def create_sliding_windows(features, targets, window_size, target_size, step_size):
    X, y = [], []

    for i in range(0, len(features) - window_size - target_size + 1, step_size):
        X.append(features.iloc[i:i + window_size].values)
        y.append(targets.iloc[i + window_size:i + window_size + target_size].values)

    return np.array(X), np.array(y)

In [33]:
#  TODO: Linear interpolation for nulls (idk if we have nulls)

In [34]:
#  Drop table if any column missing
if len(df.columns) < 43:
    print("yikes! having missing data is quite embarassing")
else:
    print("lets go")

#  Scale data with minimax
columns_non_time = df.columns[1:]
scaler_minmax = MinMaxScaler()
df[columns_non_time] = scaler_minmax.fit_transform(df[columns_non_time])

lets go


In [35]:
#  Resample data

df = resample_data(df, target_fs=100)

#  Apply butterworth filter
df = apply_filter(df, emg_columns)

#  Apply notch filter
df = apply_notch(df, emg_columns)

In [36]:
left_sensors, left_emg, right_sensors, right_emg = split_dataframe(df)

In [37]:
print(left_sensors.shape)
print(left_emg.shape)
print(right_sensors.shape)
print(right_emg.shape)

(31400, 18)
(31400, 3)
(31400, 18)
(31400, 3)


In [38]:
window_size = 10
step_size = 1
target_size = 5

X_left, y_left = create_sliding_windows(left_sensors, right_emg, window_size, target_size, step_size)
X_right, y_right = create_sliding_windows(right_sensors, left_emg, window_size, target_size, step_size)

print("X_left shape:", X_left.shape)
print("y_right shape:", y_right.shape)
print("X_right shape:", X_right.shape)
print("y_left shape:", y_left.shape)

X_left shape: (31386, 10, 18)
y_right shape: (31386, 5, 3)
X_right shape: (31386, 10, 18)
y_left shape: (31386, 5, 3)


In [39]:
#  Combine data to generalize
X_combined = np.concatenate([X_left, X_right], axis=0)
y_combined = np.concatenate([y_left, y_right], axis=0)

#  Add side indicator (0 = left sensors, 1 = right sensors)
side_indicator = np.concatenate([np.zeros(len(X_left)), np.ones(len(X_right))])
side_indicator = side_indicator.reshape(-1, 1, 1).repeat(window_size, axis=1)

#  Combine sensor data with side indicator
X_combined_with_indicator = np.concatenate([X_combined, side_indicator], axis=2)


checkpoint = ModelCheckpoint("model.h5", save_best_only=True, monitor='val_loss', mode='min', verbose=1)


#  Generalized LSTM
model = Sequential()
model.add(Input(shape=(window_size, 19)))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))  #  Stop overfitting
model.add(Dense(units=target_size * 3))  # Output 3 EMG values
model.add(Reshape((target_size, 3)))     # Shape: (samples, target_size, 3)

model.compile(optimizer='adam', loss='mse')

#  Train on combined data
model.fit(X_combined_with_indicator, 
          y_combined, epochs=20, 
          batch_size=32, 
          validation_split=0.2, 
          callbacks=[checkpoint])

predictions = model.predict(X_combined_with_indicator)

#  Predict right EMG from left sensors
X_new_left, _ = create_sliding_windows(left_sensors, right_emg, window_size, target_size, step_size)
side_indicator_new = np.zeros((X_new_left.shape[0], window_size, 1))  # 0 for left
X_new_with_indicator = np.concatenate([X_new_left, side_indicator_new], axis=2)
pred_right_emg = model.predict(X_new_with_indicator)
#  Predict left EMG from right sensors
X_new_right, _ = create_sliding_windows(right_sensors, left_emg, window_size, target_size, step_size)
side_indicator_new = np.ones((X_new_right.shape[0], window_size, 1))  # 1 for right
X_new_with_indicator = np.concatenate([X_new_right, side_indicator_new], axis=2)
pred_left_emg = model.predict(X_new_with_indicator)

Epoch 1/20
1569/1570 [============================>.] - ETA: 0s - loss: 0.0060
Epoch 1: val_loss improved from inf to 0.00005, saving model to model.h5
1570/1570 [==============================] - 24s 11ms/step - loss: 0.0060 - val_loss: 4.9149e-05
Epoch 2/20
1566/1570 [============================>.] - ETA: 0s - loss: 7.4718e-04
Epoch 2: val_loss improved from 0.00005 to 0.00003, saving model to model.h5
1570/1570 [==============================] - 18s 11ms/step - loss: 7.4653e-04 - val_loss: 3.4598e-05
Epoch 3/20
1569/1570 [============================>.] - ETA: 0s - loss: 2.2563e-04
Epoch 3: val_loss improved from 0.00003 to 0.00003, saving model to model.h5
1570/1570 [==============================] - 17s 11ms/step - loss: 2.2560e-04 - val_loss: 2.6016e-05
Epoch 4/20
1569/1570 [============================>.] - ETA: 0s - loss: 8.4143e-05
Epoch 4: val_loss improved from 0.00003 to 0.00001, saving model to model.h5
1570/1570 [==============================] - 17s 11ms/step - loss: 8.

KeyboardInterrupt: 

In [40]:
model.save('gait_model.h5')

In [73]:
#  TODO: Data augmentation ?? 

In [75]:
#  TODO: Evaluate performance of different filters